# Notebook to run basic `scanpy` QC and doublet detection with `scrublet` for SRA Project - PRJEB39602 -D3

- **Developed by**: Srivalli Kolla

- **Created date** : 14 October, 2024

- **Modification date** : 05 November, 2024

- **Würzburg Institute for Systems Immunology & Julius-Maximilian-Universität Würzburg**

Env : Scanpy(Python 3.12.4)

# Import Packages

In [14]:
import anndata
import logging
import anndata as ad
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sb
import scrublet as scr
import os
import time
import matplotlib.pyplot as plt
from statsmodels.robust.scale import mad as median_abs_deviation
from matplotlib import colors
from matplotlib import rcParams

In [15]:
sc.settings.verbosity = 3
sc.logging.print_versions()
sc.settings.set_figure_params(dpi = 160, color_map = 'RdPu', dpi_save = 180, vector_friendly = True, format = 'svg')
timestamp = time.strftime("%d_%m_%Y")

-----
anndata     0.10.8
scanpy      1.10.2
-----
PIL                 10.3.0
array_api_compat    1.7.1
asttokens           NA
attr                23.2.0
cffi                1.16.0
colorama            0.4.6
comm                0.2.2
cycler              0.12.1
cython_runtime      NA
dateutil            2.9.0.post0
debugpy             1.8.2
decorator           5.1.1
defusedxml          0.7.1
distutils           3.12.4
django              5.0.6
executing           2.0.1
h5py                3.11.0
igraph              0.11.5
ipykernel           6.29.5
ipython_genutils    0.2.0
ipywidgets          8.1.3
jedi                0.19.1
joblib              1.4.2
kiwisolver          1.4.5
legacy_api_wrap     NA
leidenalg           0.10.2
llvmlite            0.43.0
louvain             0.8.2
matplotlib          3.8.4
matplotlib_inline   0.1.7
mpl_toolkits        NA
natsort             8.4.0
numba               0.60.0
numexpr             2.10.1
numpy               1.26.4
packaging           24.1
pandas 

# Import sample_names

In [16]:
path = '/mnt/LaCIE/skolla/sc-heart-consortium/ncbi-sra/h5ad_files/'
files = ['ERR6449986_GeneFull_raw.h5ad',
'ERR7423245_GeneFull_raw.h5ad',
'ERR7423246_GeneFull_raw.h5ad',
'ERR7423472_GeneFull_raw.h5ad',
'ERR6449756_GeneFull_raw.h5ad',
'ERR6449758_GeneFull_raw.h5ad',
'ERR6449761_GeneFull_raw.h5ad',
'ERR6449775_GeneFull_raw.h5ad',
'ERR6449954_GeneFull_raw.h5ad',
'ERR6449985_GeneFull_raw.h5ad',
'ERR7423243_GeneFull_raw.h5ad',
'ERR7423262_GeneFull_raw.h5ad',
'ERR7423473_GeneFull_raw.h5ad',
'ERR6449755_GeneFull_raw.h5ad',
'ERR6449757_GeneFull_raw.h5ad',
'ERR6449759_GeneFull_raw.h5ad',
'ERR6449760_GeneFull_raw.h5ad',
'ERR6449762_GeneFull_raw.h5ad',
'ERR6449764_GeneFull_raw.h5ad',
'ERR6449770_GeneFull_raw.h5ad',
'ERR6449984_GeneFull_raw.h5ad',
'ERR6450000_GeneFull_raw.h5ad',
'ERR7423251_GeneFull_raw.h5ad',
'ERR7423441_GeneFull_raw.h5ad',
'ERR7423471_GeneFull_raw.h5ad',
'ERR7423487_GeneFull_raw.h5ad',
'ERR7423242_GeneFull_raw.h5ad',
'ERR7423244_GeneFull_raw.h5ad',
'ERR7423247_GeneFull_raw.h5ad',
'ERR7423248_GeneFull_raw.h5ad',
'ERR7423249_GeneFull_raw.h5ad',
'ERR7423257_GeneFull_raw.h5ad',]

In [17]:
adata_combined = None
vars_combined = []

for file in files:
    file_path = os.path.join(path, file)
    
    try:

        adata = sc.read_h5ad(file_path)

        sample_name = os.path.basename(file_path).split('.')[0]
        adata.obs['sample_name'] = sample_name


        sc.pp.filter_cells(adata, min_counts=10)
        sc.pp.filter_genes(adata, min_counts=10)


        adata.var_names = adata.var_names.str.split('.').str[0]
        adata.var_names = [f"{name}_{sample_name}" for name in adata.var_names]

        vars_combined.extend(adata.var_names)


        if adata_combined is None:
            adata_combined = adata
        else:
            adata_combined = sc.concat([adata_combined, adata], join='inner', index_unique='-')

        print(f"Successfully read and concatenated: {file}")

    except Exception as e:
        print(f"Error reading {file}: {e}")

unique_var_names = pd.Series(vars_combined).unique()
adata_combined.var_names = unique_var_names[:adata_combined.n_vars] 
adata_combined

filtered out 654888 cells that have less than 10 counts
filtered out 43557 genes that are detected in less than 10 counts
Successfully read and concatenated: ERR6449986_GeneFull_raw.h5ad
filtered out 658904 cells that have less than 10 counts
filtered out 42572 genes that are detected in less than 10 counts
Successfully read and concatenated: ERR7423245_GeneFull_raw.h5ad
filtered out 660593 cells that have less than 10 counts
filtered out 49184 genes that are detected in less than 10 counts
Successfully read and concatenated: ERR7423246_GeneFull_raw.h5ad
filtered out 658400 cells that have less than 10 counts
filtered out 41647 genes that are detected in less than 10 counts
Successfully read and concatenated: ERR7423472_GeneFull_raw.h5ad
filtered out 675379 cells that have less than 10 counts
filtered out 48396 genes that are detected in less than 10 counts
Successfully read and concatenated: ERR6449756_GeneFull_raw.h5ad
filtered out 658904 cells that have less than 10 counts
filtered 

AnnData object with n_obs × n_vars = 2285452 × 0
    obs: 'sample_name', 'n_counts'
    layers: 'spliced'

In [18]:
adata_combined

AnnData object with n_obs × n_vars = 2285452 × 0
    obs: 'sample_name', 'n_counts'
    layers: 'spliced'

In [19]:
adata_combined.obs

,sample_name,n_counts
AAACCTGAGAAAGTGG-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0,ERR6449986_GeneFull_raw,23
AAACCTGAGAACAATC-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0,ERR6449986_GeneFull_raw,42
AAACCTGAGAACTGTA-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0,ERR6449986_GeneFull_raw,38
AAACCTGAGACAGAGA-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0,ERR6449986_GeneFull_raw,45
AAACCTGAGACTTGAA-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0,ERR6449986_GeneFull_raw,57
...,...,...
TTTGTCATCTTAGAGC-1,ERR7423257_GeneFull_raw,19
TTTGTCATCTTGAGGT-1,ERR7423257_GeneFull_raw,36
TTTGTCATCTTGGGTA-1,ERR7423257_GeneFull_raw,30
TTTGTCATCTTGTACT-1,ERR7423257_GeneFull_raw,35


In [20]:
adata_combined.var

""


In [21]:
adata_combined.obs['sample_name'].value_counts()

sample_name
ERR6449761_GeneFull_raw    93650
ERR7423248_GeneFull_raw    93650
ERR7423244_GeneFull_raw    93167
ERR6449757_GeneFull_raw    93167
ERR7423471_GeneFull_raw    84013
ERR6449984_GeneFull_raw    84013
ERR6449986_GeneFull_raw    82392
ERR7423473_GeneFull_raw    82392
ERR7423242_GeneFull_raw    81656
ERR6450000_GeneFull_raw    79659
ERR7423487_GeneFull_raw    79659
ERR6449985_GeneFull_raw    78880
ERR7423472_GeneFull_raw    78880
ERR6449758_GeneFull_raw    78376
ERR7423245_GeneFull_raw    78376
ERR6449770_GeneFull_raw    78372
ERR7423257_GeneFull_raw    78372
ERR6449760_GeneFull_raw    78257
ERR7423247_GeneFull_raw    78257
ERR6449954_GeneFull_raw    78215
ERR7423441_GeneFull_raw    78215
ERR7423246_GeneFull_raw    76687
ERR6449759_GeneFull_raw    76687
ERR6449764_GeneFull_raw    74379
ERR7423251_GeneFull_raw    74379
ERR6449775_GeneFull_raw    71988
ERR7423243_GeneFull_raw    61901
ERR6449756_GeneFull_raw    61901
ERR6449762_GeneFull_raw    27956
ERR7423249_GeneFull_raw    2795

## Doublet score prediction

In [22]:
scrub = scr.Scrublet(adata_combined.X)

doublet_scores, predicted_doublets = scrub.scrub_doublets()
            
adata_combined.obs['doublet_scores'] = doublet_scores
adata_combined.obs['predicted_doublets'] = predicted_doublets

Preprocessing...


/home/skolla/miniforge3/envs/scanpy/lib/python3.12/site-packages/scrublet/helper_functions.py:321: RuntimeWarning: invalid value encountered in divide
  w.setdiag(float(target_total) / tots_use)


IndexError: index -1 is out of bounds for axis 0 with size 0

In [ ]:
adata_combined.obs

### Checking the count and percentage of Doublets - sample_name level

In [ ]:
doub_tab = pd.crosstab(adata_combined.obs['sample_name'],adata_combined.obs['predicted_doublets'])
doub_tab.sum()

In [ ]:
true_doublets = adata_combined.obs['predicted_doublets'] == True
true_doublets_count = true_doublets.sum()

true_doublets_percentage = (true_doublets_count / len(adata_combined.obs)) * 100

true_doublets_count ,true_doublets_percentage

### Saving raw data

In [ ]:
sample_name_object = adata_combined.copy()
sample_name_object

## Compute QC stats

In [ ]:
sample_name_object.shape

### Labelling Mt and Ribo genes

In [ ]:
sample_name_object.var

In [ ]:
sample_name_object.var.index = sample_name_object.var.index.str.split('_').str[0]
sample_name_object.var

In [ ]:
sample_name_object.var['ensembl'] = sample_name_object.var.index
sample_name_object.var 

### Ensembl annotations

In [18]:
annot = sc.queries.biomart_annotations(
        "hsapiens",
        ["ensembl_gene_id", "external_gene_name", "start_position", "end_position", "chromosome_name"],
    ).set_index("ensembl_gene_id")

In [ ]:
annot.head()

In [ ]:
sample_name_object.var

In [ ]:
sample_name_object.var['gene_name'] = sample_name_object.var.index.map(annot['external_gene_name'])
sample_name_object.var.index =sample_name_object.var['gene_name'] 
sample_name_object.var

In [ ]:
sample_name_object.var['mt'] = sample_name_object.var_names.str.startswith('MT-') 
sample_name_object.var['ribo'] = sample_name_object.var_names.str.startswith(("RPS","RPL"))
sample_name_object.var

In [ ]:
ribo_counts = sample_name_object.var['ribo'].value_counts()

mt_counts = sample_name_object.var['mt'].value_counts()

print("Counts of Ribosomal (ribo) Genes:")
print("False:", ribo_counts.get(False, 0))
print("True:", ribo_counts.get(True, 0))
print("\nCounts of Mitochondrial (mt) Genes:")
print("False:", mt_counts.get(False, 0))
print("True:", mt_counts.get(True, 0))

In [ ]:
sample_name_object.var['mt'] = sample_name_object.var['mt'].fillna(False)
sample_name_object.var['ribo'] = sample_name_object.var['ribo'].fillna(False)

### Calculating QC metrics per cell

In [25]:
sc.pp.calculate_qc_metrics(sample_name_object,qc_vars = ['mt','ribo'],inplace = True)

In [ ]:
sample_name_object

## Sex covariate analysis

### Chr Y genes calculation

In [27]:
sample_name_object.var['gene_name'] = sample_name_object.var['ensembl'].map(annot['external_gene_name'])
sample_name_object.var['chromosome'] = sample_name_object.var['ensembl'].map(annot['chromosome_name'])

In [ ]:
sample_name_object.var

In [ ]:
chrY_genes = sample_name_object.var['chromosome'] == "Y"
chrY_genes

In [ ]:
sample_name_object.obs['percent_chrY'] = np.sum(
    sample_name_object[:, chrY_genes].X, axis = 1) / np.sum(sample_name_object.X, axis = 1) * 100

In [ ]:
sample_name_object

### XIST counts

In [ ]:
sample_name_object.var_names

## Calculate cell cycle scores

### Downloading the list of cell cycle genes

In [33]:
!if [ ! -f ../ncbi_sra/data/regev_lab_cell_cycle_genes.txt ]; then curl -o ../ncbi_sra/data/regev_lab_cell_cycle_genes.txt https://raw.githubusercontent.com/theislab/scanpy_usage/master/180209_cell_cycle/data/regev_lab_cell_cycle_genes.txt; fi

### Marking cell cycle genes

#### Steps followed

1. Loading genes and captilizing 
2. Printing the length of cell cycle genes list
3. Split genes into 2 lists (#First 43 genes,#Gene 43 to end)
4. Filtering cell cycle genes only if present in processed_gene_names
5. Print the list of cell cycle genes observed in our data

In [ ]:
cell_cycle_genes = [x.strip() for x in open('../ncbi_sra/data/regev_lab_cell_cycle_genes.txt')]
#cell_cycle_genes = [gene.capitalize() for gene in cell_cycle_genes]
print(len(cell_cycle_genes))

s_genes = cell_cycle_genes[:43]
g2m_genes = cell_cycle_genes[43:]

cell_cycle_genes = [x for x in cell_cycle_genes if x in sample_name_object.var_names]
print(len(cell_cycle_genes))

In [ ]:
cell_cycle_genes

### Creating basic anndata and normalization for cell cycle score calculation

In [ ]:
adata_combined_log = anndata.AnnData(X = sample_name_object.X,  var = sample_name_object.var, obs = sample_name_object.obs)
sc.pp.normalize_total(adata_combined_log, target_sum = 1e6, exclude_highly_expressed = True)
sc.pp.log1p(adata_combined_log)

### Cell cycle score calculation

In [37]:
adata_combined_log.var_names = adata_combined_log.var_names.astype(str)
adata_combined_log.var_names_make_unique()

In [ ]:
sc.tl.score_genes_cell_cycle(adata_combined_log, s_genes = s_genes, g2m_genes = g2m_genes)

sample_name_object.obs['S_score'] = adata_combined_log.obs['S_score']
sample_name_object.obs['G2M_score'] = adata_combined_log.obs['G2M_score']
sample_name_object.obs['phase'] = adata_combined_log.obs['phase']

sample_name_object

In [ ]:
cell_cycle_counts = sample_name_object.obs['phase'].value_counts()

cell_cycle_counts

In [ ]:
sb.countplot(data=sample_name_object.obs, x='phase')

## Data visualization

In [ ]:
variables = 'n_genes_by_counts', 'total_counts', 'doublet_scores', 'G2M_score', 'S_score'

for var in variables:

    fig, ax = plt.subplots(figsize=(12, 6), ncols=2, gridspec_kw={'width_ratios': [4, 1]})

    sb.violinplot(data=sample_name_object.obs,x = 'sample_name' , y=var, ax=ax[0])
   
    medians = sample_name_object.obs.groupby('sample_name')[var].median()

    for sample_name, median in medians.items():
        ax[0].text(sample_name, median, f'{median:.2f}', ha='center', va='bottom', color='black', fontsize=10)
    
    ax[0].set_title(f'Violin Plot of {var} by sample_name - Before filtering')
    ax[0].set_xlabel('sample_name')
    ax[0].set_ylabel(var)
    ax[0].tick_params(axis='x', rotation=45)

    median_df = pd.DataFrame({'sample_name': medians.index, 'Median': medians.values})

    ax[1].axis('off')
    ax[1].table(cellText=median_df.values, colLabels=median_df.columns, loc='center')
    ax[1].set_title('Median Values')
    
    plt.tight_layout()
    plt.show()


### Visualization of qc metrics

In [ ]:
variables = ['pct_counts_mt', 'pct_counts_ribo']

sb.violinplot(data=sample_name_object.obs[variables])
plt.xticks(rotation=45)
plt.title(f'Mt and Ribo percentages - Before filtering')

In [ ]:
plt.figure(figsize=(10, 6))
sb.scatterplot(data=sample_name_object.obs, x='total_counts', y='n_genes_by_counts' , alpha = 0.4, s=4)
#plt.xticks(range(0, int(max(sample_name_object.obs['total_counts'])) + 1, 3000),rotation=45, fontsize = 10)
#plt.yticks(range(0, int(max(sample_name_object.obs['n_genes_by_counts'])) + 1, 1000),fontsize = 10)
plt.title(f'Counts vs Genes - Before filtering')
plt.show()

### Filtering based on QC metrics

In [ ]:
filtered_object = sample_name_object[sample_name_object.obs['n_genes_by_counts'] > 10]
filtered_object = filtered_object[filtered_object.obs['n_genes_by_counts'] < 400]

filtered_object = filtered_object[filtered_object.obs['total_counts'] > 10]
filtered_object = filtered_object[filtered_object.obs['total_counts'] < 400]

filtered_object = filtered_object[filtered_object.obs['pct_counts_mt'] < 60]
filtered_object = filtered_object[filtered_object.obs['pct_counts_ribo'] < 20]

filtered_object = filtered_object[filtered_object.obs['doublet_scores'] < 0.35]

filtered_object

In [ ]:
variables = ['pct_counts_mt', 'pct_counts_ribo']

sb.violinplot(data=filtered_object.obs[variables])
plt.xticks(rotation=45)
plt.title(f'Mt and Ribo percentages - After filtering')

In [ ]:
sb.set(style = "whitegrid")
covariate_to_visualize = 'total_counts'

plt.figure(figsize = (10, 6))
sb.histplot(data = filtered_object.obs, x = covariate_to_visualize, stat = 'count', common_norm = False)
plt.xlabel(covariate_to_visualize)
plt.ylabel('Abundance')
plt.title(f'Abundance Plot of {covariate_to_visualize} by sample_name - After filtering')
plt.show()

In [ ]:
sb.set(style = "whitegrid")
covariate_to_visualize = 'n_genes_by_counts'

plt.figure(figsize = (10, 6))
sb.histplot(data = filtered_object.obs, x = covariate_to_visualize, stat = 'count', common_norm = False)
plt.xlabel(covariate_to_visualize)
plt.ylabel('Abundance')
plt.title(f'Abundance Plot of {covariate_to_visualize} by sample_name - After filtering')
plt.show()

In [ ]:
variables = 'n_genes_by_counts', 'total_counts', 'pct_counts_mt', 'pct_counts_ribo', 'doublet_scores', 'G2M_score', 'S_score' 

for var in variables:

    fig, ax = plt.subplots(figsize=(12, 6), ncols=2, gridspec_kw={'width_ratios': [4, 1]})

    sb.violinplot(data=filtered_object.obs, x='sample_name', y=var, ax=ax[0])
   
    medians = filtered_object.obs.groupby('sample_name')[var].median()

    for sample_name, median in medians.items():
        ax[0].text(sample_name, median, f'{median:.2f}', ha='center', va='bottom', color='black', fontsize=10)
    
    ax[0].set_title(f'Violin Plot of {var} by sample_name - After filtering')
    ax[0].set_xlabel('sample_name')
    ax[0].set_ylabel(var)
    ax[0].tick_params(axis='x', rotation=45)

    median_df = pd.DataFrame({'sample_name': medians.index, 'Median': medians.values})

    ax[1].axis('off')
    ax[1].table(cellText=median_df.values, colLabels=median_df.columns, loc='center')
    ax[1].set_title('Median Values')
    
    plt.tight_layout()
    plt.show()

## Data Export

In [ ]:
filtered_object.raw = filtered_object.copy()

filtered_object.layers['raw_counts'] = filtered_object.X.copy()

filtered_object.layers["sqrt_norm"] = np.sqrt(
    sc.pp.normalize_total(filtered_object, inplace = False)["X"]
)

filtered_object

In [ ]:
filtered_object.obs['sample_name'].value_counts()

In [ ]:
filtered_object.var.dtypes

In [ ]:
filtered_object.var['mt'].value_counts()

In [53]:
filtered_object.var['mt'] = filtered_object.var['mt'].astype(str)

In [ ]:
print(filtered_object.var.dtypes)

In [ ]:
filtered_object.var

In [ ]:
filtered_object.var = filtered_object.var.rename(columns={'gene_name': 'gene_symbol'})
filtered_object.var = filtered_object.var.reset_index()
filtered_object.var

In [57]:
filtered_object.raw.var.index.name = 'gene_id'  

In [58]:
filtered_object.write_h5ad(f'../ncbi_sra/data/PRJEB59734_sra_filtered_sk_{timestamp}.h5ad')